In [1]:
import pandas as pd
from geopy import Nominatim
import numpy as np

In [2]:
superfund_df = pd.read_csv('resources/superfund_sites.csv')

In [3]:
superfund_df.head()

,RG,SITE_ID,SITE_EPA_ID,SITE_NAME,SITE_STRT_ADRS1,SITE_STRT_ADRS2,SITE_CITY_NAME,SITE_STATE,SITE_ZIP_CODE,SITE_CONG_DISTRICT,SITE_CNTY_NAME,SITE_FIPS_CODE,LATITUDE,LONGITUDE,ALT_AGREEMENT,NPL,FF,NON_NPL_STATUS
0,4,406154,AL0000047829,WEST END LANDFILL,HWY 202 AND ADAMS ST,NaN,ANNISTON,AL,36202,03,CALHOUN,1015.0,NaN,NaN,N,Not on the NPL,N,N
1,4,406158,AL0000059147,ETOWAH COUNTY (RIDGEVILLE) LANDFILL,NOBLE HILL ROAD,NaN,RIDGEVILLE,AL,35954,04,ETOWAH,1055.0,NaN,NaN,N,Not on the NPL,N,N
2,4,406176,AL0000132662,FUSSY HILL DRUM DUMP,461 FUSSY HILL ROAD,NaN,NEW MARKET,AL,35761,05,MADISON,1089.0,NaN,NaN,N,Not on the NPL,N,N
3,4,406800,AL0000301747,JONES PROPERTY,595 GRADY STREET,NaN,MONTGOMERY,AL,36108,NaN,MONTGOMERY,1101.0,NaN,NaN,N,Not on the NPL,N,N
4,4,406321,AL0001024793,ILCO SATELLITE SITES,INTERSECTION OF HWY 78 AND HWY 411,NaN,LEEDS,AL,35094,06,JEFFERSON,1073.0,NaN,NaN,N,Not on the NPL,N,N


In [4]:
# Replaces "%26" with "&"; data from EPA Databases replaces "&" with "%26"
def replace_amp(string):
    return string.replace("%26", "&")

In [5]:
# Change Datatypes
superfund_df['SITE_STRT_ADRS1'] = superfund_df['SITE_STRT_ADRS1'].astype(str)
superfund_df['SITE_CITY_NAME'] = superfund_df['SITE_CITY_NAME'].astype(str)
superfund_df['SITE_STATE'] = superfund_df['SITE_STATE'].astype(str)
superfund_df['SITE_ZIP_CODE'] = superfund_df['SITE_ZIP_CODE'].astype(str)

In [6]:
# Replaces "%26" with "&"; data from EPA Databases replaces "&" with "%26"
superfund_df['SITE_NAME']=superfund_df['SITE_NAME'].apply(replace_amp)
superfund_df['SITE_STRT_ADRS1']=superfund_df['SITE_STRT_ADRS1'].apply(replace_amp)

In [7]:
# Create Full Address Column to get lat/longs from
superfund_df['FULL_ADDRESS'] = superfund_df['SITE_STRT_ADRS1'] + ", " + superfund_df['SITE_CITY_NAME'] + ", " + superfund_df['SITE_STATE'] + ", " + superfund_df['SITE_ZIP_CODE']

In [13]:
superfund_df['FULL_ADDRESS']

0                HWY 202 AND ADAMS ST, ANNISTON, AL, 36202
1                   NOBLE HILL ROAD, RIDGEVILLE, AL, 35954
2               461 FUSSY HILL ROAD, NEW MARKET, AL, 35761
3                  595 GRADY STREET, MONTGOMERY, AL, 36108
4        INTERSECTION OF HWY 78 AND HWY 411, LEEDS, AL,...
                               ...                        
13162    Hartville Hwy 270 near the town of Hartville, ...
13163    Intersection of Wyoming 487 and Shriley Ridge ...
13164    Intersection of W. 5th Street and Thomes Avenu...
13165    Yellowstone National Park, TOWER JUNCTION, WY,...
13166    FIFTEEN MILES NORTHWEST OF CHEYENNE, CHEYENNE,...
Name: FULL_ADDRESS, Length: 13167, dtype: object

In [34]:
# Returns lat/long from address
def lat_long_locator(address):
    global counter
    try:
        geolocator = Nominatim(user_agent="my_user_agent")
        location = geolocator.geocode(address)
        coords = (location.latitude, location.longitude)
        counter += 1
        if counter % 1000 == 0:
            print(f'{counter} items completed')
        return(coords)
    except:
        counter += 1
        if counter % 1000 == 0:
            print(f'{counter} items completed')
        return(np.NaN)

In [18]:
lat_long_locator('461 FUSSY HILL ROAD, NEW MARKET, AL, 35761')

(34.843856851295506, -86.42467197859389)

In [35]:
df_test.drop(columns = "COORDINATES", inplace = True)

KeyError: "['COORDINATES'] not found in axis"

In [14]:
df_test = superfund_df.iloc[0:25]

In [30]:
df_test["COORDINATES"]

0                                          NaN
1                                          NaN
2     (34.843856851295506, -86.42467197859389)
3      (32.36915695918368, -86.31928473469388)
4                                          NaN
5             (32.3747253, -86.31246495527151)
6                                          NaN
7                                          NaN
8                                          NaN
9                      (31.193754, -88.173334)
10                                         NaN
11     (33.11663487815802, -86.85684992008537)
12                     (30.668933, -88.046487)
13                                         NaN
14            (34.6860037, -86.67197170092993)
15                                         NaN
16     (31.17712633615353, -85.39836949672612)
17     (40.69926087903226, -74.09123539112903)
18      (32.9466546556476, -85.98235793021082)
19                                         NaN
20                   (42.9838158, -90.6488432)
21           

In [ ]:
# Retreiving address for each site
# Creating temporary copy of dataframe to avoid "SettingWithCopy" warning
counter = 0
superfund_temp = superfund_df.copy()
superfund_df["COORDINATES"] = superfund_temp["FULL_ADDRESS"].apply(lat_long_locator)